## Noise

- Measure noise from continuum images

We measure the noise of images from 24 frequency intervals, between 84 MHz and 217 GHz.

In [ ]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable
from astropy.table import Table

fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/'
regDir = rootDir+'inIms/regions/'
imDir = rootDir+'inIms/'
inTabs = rootDir+'inTabs/'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir

aut = 'FgIndCv'

tableName = inTabs+'inputFluxTable'+aut+'.tbl'
tableFileNames= ascii.read(tableName,format='csv')

outTable = inTabs+'noiseFluxInt'+aut+'Ind.tbl'
#frList = [ 84.0,114.5,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
#      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,100000.0,143000.0, 217000.0]
#frList = [ 84.0,114.5,155,200.5, 320.0,1029.9794921,1440.5292968,1500.0,
#      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0,
#          30000.0,44000.0,70000.0,100000.0,143000.0, 217000.0,353000.0,545000.0,857000.0]
frList = [200.5, 320.0,1029.9794921,1440.5292968,1500.0,
      5700.0,5900.0,6100.0,6300.0,6500.0,6700.0,6874.0]
print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''
print tableFileNames.columns[0]

In [ ]:
totReg = regDir+'ToT.reg'
elReg = regDir+'EL.reg'
wlReg = regDir+'WL.reg'
anReg = regDir+'an_back.reg'
regions = [totReg, elReg, wlReg, wlReg]

errPerc=  np.array(tableFileNames.columns[2],dtype=float)

fr=[]
rmsArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
noiseArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
backArray= np.zeros([len(tableFileNames.columns[0])])

cutoff= np.zeros([len(tableFileNames.columns[0])])

for i in xrange(0,len(tableFileNames.columns[0])):

    fileName = imDir+tableFileNames.columns[0][i]

    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)

    heads=fint.cleanHead(heads)

    if not 'CDELT2' in heads:
        heads['CDELT2'] = heads['CD2_2']
        heads['CDELT1'] = heads['CD1_1']
    #if 'Frequency' in tableFileNames.dtype.names:
    #    freq = float(tableFileNames.columns[1][i])*1e6
    #    fr.append(freq)
    #elif 'FREQ' in heads:
    #    freq = float(heads['FREQ'])
    #    fr.append(freq)
    #elif 'CRVAL3' in heads:
    #    freq = float(heads['CRVAL3'])
    #    fr.append(freq)
    #else:
    freq = frList[i]*1e6
    fr.append(freq)
        
    beamArea = 2*np.pi*float(heads['BMAJ'])*3600./2.35482*float(heads['BMIN'])*3600./2.35482
    pixArea = -float(heads['CDELT2']*3600.)*float(heads['CDELT1']*3600.)
    numPixBeam= beamArea/pixArea   
    
    for j in xrange (0,len(regions)):
        
        if j==0:
            background, rms, pixels = fint.noiseReg(datas,heads,regions[j])
            background2, rms1, pixels1 = fint.noiseReg(datas,heads,anReg)

        else:
            backround, rmsEL, pixels = fint.noiseReg(datas,heads,regions[j])
        
        noiseArray[j,i] = rms*pixels/numPixBeam
    
    backArray[i] = background2    
    
    rmsArray[:,i] = rms

#for i in xrange(15,len(tableFileNames)):
    
#        fileName = imDir+tableFileNames.columns[0][i]
#        print fileName

#        datas,heads = fint.openFile(fileName)
#        datas=np.squeeze(datas)
#        heads=fint.cleanHead(heads)
#        if not 'CDELT2' in heads:
#            heads['CDELT2'] = heads['CD2_2']
#            heads['CDELT1'] = heads['CD1_1']
        #if 'Frequency' in tableFileNames.dtype.names:
        #    freq = float(tableFileNames.columns[1][i])*1e6
        #    fr.append(freq)
        #elif 'FREQ' in heads:
        #    freq = float(heads['FREQ'])
        #    fr.append(freq)
        #elif 'CRVAL3' in heads:
        #    freq = float(heads['CRVAL3'])
        #    fr.append(freq)
        #else:
#        freq = frList[i]*1e6
#        fr.append(freq)
#        beamArea = 2*np.pi*float(heads['BMAJ'])*3600./2.35482*float(heads['BMIN'])*3600./2.35482
#        pixArea = -float(heads['CDELT2']*3600.)*float(heads['CDELT1']*3600.)
#        numPixBeam= beamArea/pixArea   
#        for j in xrange (0,len(regions)):
            
#            if j==0:
#                background, rms, pixels = fint.noiseMultiReg(datas,heads,regDir+'noiseReg/')
#                background1, rms1, pixels1 = fint.noiseReg(datas,heads,regions[0])
#                background2, rms1, pixels1 = fint.noiseReg(datas,heads,anReg)
#                print pixels, pixels1
#                rmsArray[j,i] = rms/np.sqrt(pixels)
#                noiseArray[0,i] = rms/numPixBeam
#                print noiseArray[0,i]
    
#            elif j==1:
#                background, rms, pixels = fint.noiseMultiReg(datas,heads,regDir+'noiseReg/Eregions/')
#                background1, rms1, pixels1 = fint.noiseReg(datas,heads,regions[1])
#                print pixels, pixels1
#                noiseArray[j,i] = rms/numPixBeam

#                rmsArray[j,i] = rms/np.sqrt(pixels)
                
#            elif j==2:
#                background, rms, pixels = fint.noiseMultiReg(datas,heads,regDir+'noiseReg/Wregions/')
#                noiseArray[j,i] = rms     /numPixBeam           
#                rmsArray[j,i] = rms/np.sqrt(pixels)
                
#            elif j==3:
#                background, rms, pixels = fint.noiseMultiReg(datas,heads,regDir+'noiseReg/Cregions/')
#                noiseArray[j,i] = rms  /numPixBeam
#                rmsArray[j,i] = rms/np.sqrt(pixels)    
        
#        backArray[i] = background2

#print '''\t+---------+\n\t Noise measured\n\t+---------+'''
  

**Output tables**:

```
~/Projects/MFS/FornaxA/contData/noiseFluxes.tbl'

```


In [ ]:
colNames = ['#Filenames','FlCorr','ErrPerc','rmsT [Jy/pix]', 'rmsEL [Jy/pix]', 'rmsWL [Jy/pix]','rmsC [Jy/pix]',
            'noiseToT [Jy]', 'noiseEL [Jy]', 'noiseWL [Jy]', 'noiseC [Jy]', 'back [Jy/pix]']

flCorr = np.array(tableFileNames.columns[1],dtype=float)
errPerc=  np.array(tableFileNames.columns[2],dtype=float)

alldata = np.array([tableFileNames.columns[0],flCorr,errPerc,rmsArray[0,:].T,rmsArray[1,:].T,rmsArray[2,:].T,
                    rmsArray[3,:].T,noiseArray[0,:].T,noiseArray[1,:].T,noiseArray[2,:].T,noiseArray[3,:].T,
                    backArray[:].T])

tt = Table(alldata.T,names=colNames,meta={'name': 'Noise table'})

ascii.write(tt,outTable, overwrite=True)

t = PrettyTable(colNames)
t.add_row(alldata)


print t
print '''\t+---------+\n\t Tables Saved\n\t+---------+'''
